#### install yolov3 

In [1]:
!git clone https://github.com/ultralytics/yolov3
!cd yolov3; pip install -qr requirements.txt

Cloning into 'yolov3'...
remote: Enumerating objects: 10029, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 10029 (delta 3), reused 9 (delta 3), pack-reused 10017
Receiving objects: 100% (10029/10029), 9.36 MiB | 10.58 MiB/s, done.
Resolving deltas: 100% (6762/6762), done.
     |████████████████████████████████| 596 kB 18.5 MB/s 
     |████████████████████████████████| 1.8 MB 45.1 MB/s 
     |████████████████████████████████| 153 kB 44.6 MB/s 
     |████████████████████████████████| 181 kB 52.1 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 


#### download Oxford Pet Dataset 

In [8]:
!wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
!wget https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz

--2022-07-26 03:28:21--  https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 791918971 (755M) [application/x-gzip]
Saving to: ‘images.tar.gz’

images.tar.gz       100%[===================>] 755.23M  34.8MB/s    in 23s     

2022-07-26 03:28:44 (33.1 MB/s) - ‘images.tar.gz’ saved [791918971/791918971]

--2022-07-26 03:28:44--  https://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 19173078 (18M) [application/x-gzip]
Saving to: ‘annotations.tar.gz’

annotations.tar.gz  100%[===================>]  18.28M  15.3MB/s    in 1.2s    

2022-07-26 03:28:46 (15.3 M

In [10]:
!mkdir /content/dataset
!tar -xvf images.tar.gz -C /content/dataset
!tar -xvf annotations.tar.gz -C /content/dataset


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
annotations/trimaps/._pomeranian_180.png
annotations/trimaps/pomeranian_180.png
annotations/trimaps/._pomeranian_181.png
annotations/trimaps/pomeranian_181.png
annotations/trimaps/._pomeranian_182.png
annotations/trimaps/pomeranian_182.png
annotations/trimaps/._pomeranian_183.png
annotations/trimaps/pomeranian_183.png
annotations/trimaps/._pomeranian_184.png
annotations/trimaps/pomeranian_184.png
annotations/trimaps/._pomeranian_185.png
annotations/trimaps/pomeranian_185.png
annotations/trimaps/._pomeranian_186.png
annotations/trimaps/pomeranian_186.png
annotations/trimaps/._pomeranian_187.png
annotations/trimaps/pomeranian_187.png
annotations/trimaps/._pomeranian_188.png
annotations/trimaps/pomeranian_188.png
annotations/trimaps/._pomeranian_189.png
annotations/trimaps/pomeranian_189.png
annotations/trimaps/._pomeranian_19.png
annotations/trimaps/pomeranian_19.png
annotations/trimaps/._pomeranian_190.png
annotations/trimaps/pomeranian_190.png
annota

#### train valid splot 


In [11]:
!mkdir /content/ox_pet 
!cd /content/ox_pet; mkdir images; mkdir labels; 
!cd /content/ox_pet/images; mkdir train; mkdir val 
!cd /content/ox_pet/labels; mkdir train; mkdir val 

In [12]:
import torch 
import numpy as np 
import pandas as pd 
import cv2 
import os 
import sys 
import matplotlib
import matplotlib.pyplot as plt
import sklearn 


from IPython.display import Image , clear_output
from google.colab import drive 
from google.colab.patches import cv_imshow
from sklearn.model_selection import train_test_split 

In [13]:
def show_version():
    print("=========version=========")
    print(f"python : {sys.version}")
    print(f"torch : {torch.__version__}")
    print(f"numpy : {np.__version__}")
    print(f"pandas : {pd.__version__}")
    print(f"matplotlib : {matplotlib.__version__}")
    print(f"sklearn : {sklearn.__version__}")

show_version()

=========version=========
python : 3.7.13 (default, Apr 24 2022, 01:04:09) 
[GCC 7.5.0]
torch : 1.12.0+cu113
numpy : 1.21.6
pandas : 1.3.5
matplotlib : 3.2.2
sklearn : 1.0.2


In [15]:
pet_df =pd.read_csv("/content/dataset/annotations/trainval.txt", sep=" ", header=None, names=["img_name", "class_id", "etc1", "etc2"])
pet_df

,img_name,class_id,etc1,etc2
0,Abyssinian_100,1,1,1
1,Abyssinian_101,1,1,1
2,Abyssinian_102,1,1,1
3,Abyssinian_103,1,1,1
4,Abyssinian_104,1,1,1
...,...,...,...,...
3675,yorkshire_terrier_187,37,2,25
3676,yorkshire_terrier_188,37,2,25
3677,yorkshire_terrier_189,37,2,25
3678,yorkshire_terrier_18,37,2,25


In [30]:
def set_train_valid_split(list_filepath, img_dir, anno_dir, test_size =0.1):
    pet_df = pd.read_csv(list_filepath, sep=" ", header=None, names=["img_name", "class_id", "etc1", "etc2"])

    pet_df["class_name"] = pet_df["img_name"].apply(lambda x:x[:x.rfind("_")])
  

    pet_df["img_filepath"] = img_dir + pet_df["img_name"] + ".jpg"
    pet_df["anno_filepath"] = anno_dir + pet_df["img_name"] + ".xml"

    pet_df = set_remove_no_annos(pet_df)

    train_df, val_df = train_test_split(pet_df, test_size=test_size, stratify=pet_df["class_id"], random_state =2021)
    return pet_df, train_df, val_df  

def set_remove_no_annos(df):
    remove_rows = []
    for index, row in df.iterrows():
        anno_filepath = row["anno_filepath"]
        if not os.path.exists(anno_filepath):
            print(f"##### index : {index} , {anno_filepath} does not exist ti deleted")
            remove_rows.append(index)
    df = df.drop(remove_rows, axis=0, inplace=False)

    return df 




In [34]:
pet_df, train_df, val_df = set_train_valid_split("/content/dataset/annotations/trainval.txt",
                                                 "/content/dataset/images/",
                                                 "/content/dataset/annotations/xmls/",
                                                 test_size=0.1) 
  

##### index : 4 , /content/dataset/annotations/xmls/Abyssinian_104.xml does not exist ti deleted
##### index : 262 , /content/dataset/annotations/xmls/Bengal_111.xml does not exist ti deleted
##### index : 1456 , /content/dataset/annotations/xmls/samoyed_10.xml does not exist ti deleted
##### index : 2128 , /content/dataset/annotations/xmls/Bengal_175.xml does not exist ti deleted
##### index : 2395 , /content/dataset/annotations/xmls/Egyptian_Mau_14.xml does not exist ti deleted
##### index : 2402 , /content/dataset/annotations/xmls/Egyptian_Mau_156.xml does not exist ti deleted
##### index : 2427 , /content/dataset/annotations/xmls/Egyptian_Mau_186.xml does not exist ti deleted
##### index : 3177 , /content/dataset/annotations/xmls/Ragdoll_199.xml does not exist ti deleted
##### index : 3246 , /content/dataset/annotations/xmls/saint_bernard_15.xml does not exist ti deleted


In [35]:
pet_df

,img_name,class_id,etc1,etc2,class_name,img_filepath,anno_filepath
0,Abyssinian_100,1,1,1,Abyssinian,/content/dataset/images/Abyssinian_100.jpg,/content/dataset/annotations/xmls/Abyssinian_1...
1,Abyssinian_101,1,1,1,Abyssinian,/content/dataset/images/Abyssinian_101.jpg,/content/dataset/annotations/xmls/Abyssinian_1...
2,Abyssinian_102,1,1,1,Abyssinian,/content/dataset/images/Abyssinian_102.jpg,/content/dataset/annotations/xmls/Abyssinian_1...
3,Abyssinian_103,1,1,1,Abyssinian,/content/dataset/images/Abyssinian_103.jpg,/content/dataset/annotations/xmls/Abyssinian_1...
5,Abyssinian_105,1,1,1,Abyssinian,/content/dataset/images/Abyssinian_105.jpg,/content/dataset/annotations/xmls/Abyssinian_1...
...,...,...,...,...,...,...,...
3675,yorkshire_terrier_187,37,2,25,yorkshire_terrier,/content/dataset/images/yorkshire_terrier_187.jpg,/content/dataset/annotations/xmls/yorkshire_te...
3676,yorkshire_terrier_188,37,2,25,yorkshire_terrier,/content/dataset/images/yorkshire_terrier_188.jpg,/content/dataset/annotations/xmls/yorkshire_te...
3677,yorkshire_terrier_189,37,2,25,yorkshire_terrier,/content/dataset/images/yorkshire_terrier_189.jpg,/content/dataset/annotations/xmls/yorkshire_te...
3678,yorkshire_terrier_18,37,2,25,yorkshire_terrier,/content/dataset/images/yorkshire_terrier_18.jpg,/content/dataset/annotations/xmls/yorkshire_te...


In [36]:
train_df

,img_name,class_id,etc1,etc2,class_name,img_filepath,anno_filepath
305,Birman_105,7,1,3,Birman,/content/dataset/images/Birman_105.jpg,/content/dataset/annotations/xmls/Birman_105.xml
2174,Birman_171,7,1,3,Birman,/content/dataset/images/Birman_171.jpg,/content/dataset/annotations/xmls/Birman_171.xml
971,leonberger_123,20,2,14,leonberger,/content/dataset/images/leonberger_123.jpg,/content/dataset/annotations/xmls/leonberger_1...
2454,english_cocker_spaniel_15,13,2,7,english_cocker_spaniel,/content/dataset/images/english_cocker_spaniel...,/content/dataset/annotations/xmls/english_cock...
1656,Sphynx_115,34,1,12,Sphynx,/content/dataset/images/Sphynx_115.jpg,/content/dataset/annotations/xmls/Sphynx_115.xml
...,...,...,...,...,...,...,...
3087,pug_153,26,2,18,pug,/content/dataset/images/pug_153.jpg,/content/dataset/annotations/xmls/pug_153.xml
658,english_setter_111,14,2,8,english_setter,/content/dataset/images/english_setter_111.jpg,/content/dataset/annotations/xmls/english_sett...
1903,american_bulldog_153,2,2,1,american_bulldog,/content/dataset/images/american_bulldog_153.jpg,/content/dataset/annotations/xmls/american_bul...
273,Bengal_121,6,1,2,Bengal,/content/dataset/images/Bengal_121.jpg,/content/dataset/annotations/xmls/Bengal_121.xml


In [37]:
val_df

,img_name,class_id,etc1,etc2,class_name,img_filepath,anno_filepath
909,keeshond_112,19,2,13,keeshond,/content/dataset/images/keeshond_112.jpg,/content/dataset/annotations/xmls/keeshond_112...
1404,saint_bernard_108,29,2,19,saint_bernard,/content/dataset/images/saint_bernard_108.jpg,/content/dataset/annotations/xmls/saint_bernar...
3075,pomeranian_186,25,2,17,pomeranian,/content/dataset/images/pomeranian_186.jpg,/content/dataset/annotations/xmls/pomeranian_1...
2429,Egyptian_Mau_188,12,1,6,Egyptian_Mau,/content/dataset/images/Egyptian_Mau_188.jpg,/content/dataset/annotations/xmls/Egyptian_Mau...
2274,boxer_174,9,2,5,boxer,/content/dataset/images/boxer_174.jpg,/content/dataset/annotations/xmls/boxer_174.xml
...,...,...,...,...,...,...,...
751,great_pyrenees_105,16,2,10,great_pyrenees,/content/dataset/images/great_pyrenees_105.jpg,/content/dataset/annotations/xmls/great_pyrene...
3157,Ragdoll_180,27,1,9,Ragdoll,/content/dataset/images/Ragdoll_180.jpg,/content/dataset/annotations/xmls/Ragdoll_180.xml
1623,Siamese_135,33,1,11,Siamese,/content/dataset/images/Siamese_135.jpg,/content/dataset/annotations/xmls/Siamese_135.xml
1022,Maine_Coon_132,21,1,7,Maine_Coon,/content/dataset/images/Maine_Coon_132.jpg,/content/dataset/annotations/xmls/Maine_Coon_1...


In [ ]:
for index, row in pet_df.iterrows():
    print(index, row)

In [41]:
for index, row in pet_df.iterrows():
    anno_file = row["anno_filepath"]
    if not os.path.exists(anno_file):
        print(anno_file)